## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.42,19.23,54.81,85,99,5.88,overcast clouds
1,1,Bluff,NZ,-46.60,168.33,44.80,79,100,11.36,light rain
2,2,Skegness,GB,53.14,0.34,93.02,44,71,6.46,broken clouds
3,3,Bathsheba,BB,13.22,-59.52,86.61,80,20,20.71,few clouds
4,4,Grindavik,IS,63.84,-22.43,54.07,82,75,4.61,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_user_temp = int(input("What is the lowest temperature you would want at your destination? "))
max_user_temp = int(input("What is the highest temperature you would want at your destination? "))

What is the lowest temperature you would want at your destination? 75
What is the highest temperature you would want at your destination? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_user_temp)  & (city_data_df["Max Temp"] <= max_user_temp)]
temp_filtered_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Bathsheba,BB,13.22,-59.52,86.61,80,20,20.71,few clouds
9,9,Bethel,US,41.37,-73.41,77.36,91,100,9.22,overcast clouds
12,12,Arraial Do Cabo,BR,-22.97,-42.03,75.06,78,20,4.61,few clouds
13,13,Atuona,PF,-9.80,-139.03,77.97,64,72,18.72,broken clouds
15,15,Aktau,KZ,43.65,51.20,86.58,48,0,9.06,clear sky
...,...,...,...,...,...,...,...,...,...,...
682,682,Mahon,ES,39.89,4.27,84.76,58,0,6.91,clear sky
684,684,Sao Lourenco,BR,-22.12,-45.05,82.42,35,36,4.18,scattered clouds
685,685,Iquitos,PE,-3.75,-73.25,80.64,83,75,6.91,broken clouds
687,687,Pondicherry,IN,11.93,79.83,84.49,69,100,4.41,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
temp_filtered_df.isnull().count()

City_ID                258
City                   258
Country                258
Lat                    258
Lng                    258
Max Temp               258
Humidity               258
Cloudiness             258
Wind Speed             258
Current Description    258
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_filtered_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bathsheba,BB,86.61,few clouds,13.22,-59.52,
9,Bethel,US,77.36,overcast clouds,41.37,-73.41,
12,Arraial Do Cabo,BR,75.06,few clouds,-22.97,-42.03,
13,Atuona,PF,77.97,broken clouds,-9.80,-139.03,
15,Aktau,KZ,86.58,clear sky,43.65,51.20,
17,Ibra,OM,81.59,overcast clouds,22.69,58.53,
22,Moussoro,TD,87.04,light rain,13.64,16.49,
26,Sao Joao Da Barra,BR,80.69,few clouds,-21.64,-41.05,
27,Dombarovskiy,RU,77.13,clear sky,50.76,59.54,
29,Hithadhoo,MV,82.24,overcast clouds,-0.60,73.08,


In [8]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    long = row['Lng']
    params["location"] = f"{lat},{long}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json' 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_search = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_search['results'][0]['name']
    except:
        print("Hotel not found... Skipping")
        

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))